In [1]:
import geopandas as gpd
from shapely.ops import split
from shapely.geometry import LineString, MultiLineString
import pandas as pd

buffer = 5

streams_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams.shp'
manual_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/ManuallyDigitizedDitchesKrycklan.shp'
out_fp = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_{buffer}.shp'

# Load shapefiles
gdf_streams = gpd.read_file(streams_fp)
gdf_manual = gpd.read_file(manual_fp)

# Ensure same CRS
gdf_manual = gdf_manual.to_crs(gdf_streams.crs)

# --- Harmonize attribute column ---
# If manual shapefile has 'GRID_CODE', rename it to 'VALUE'
if 'GRID_CODE' in gdf_manual.columns:
    gdf_manual = gdf_manual.rename(columns={'GRID_CODE': 'VALUE'})
if 'GRID_CODE' in gdf_streams.columns:
    gdf_streams = gdf_streams.rename(columns={'GRID_CODE': 'VALUE'})

# Buffer the 5ha streams to catch near-overlaps
gdf_streams_buffer = gdf_streams.copy()
gdf_streams_buffer['geometry'] = gdf_streams_buffer.geometry.buffer(buffer)

# Function to remove overlapping segments
def remove_overlaps(line, buffered_streams):
    result = [line]
    for buf in buffered_streams.geometry:
        temp = []
        for seg in result:
            if seg.intersects(buf):
                diff = seg.difference(buf)
                # difference may return MultiLineString or LineString
                if diff.is_empty:
                    continue
                elif isinstance(diff, LineString):
                    temp.append(diff)
                elif isinstance(diff, MultiLineString):
                    temp.extend(diff.geoms)
            else:
                temp.append(seg)
        result = temp
    return result

# Apply to manual lines
manual_cleaned = []
for idx, row in gdf_manual.iterrows():
    non_overlap_segments = remove_overlaps(row.geometry, gdf_streams_buffer)
    for seg in non_overlap_segments:
        if seg.length > 0:  # discard tiny fragments if needed
            manual_cleaned.append({'geometry': seg, **row.drop('geometry')})

manual_gdf = gpd.GeoDataFrame(manual_cleaned, crs=gdf_manual.crs)

# Combine with 5haStreams
merged_gdf = pd.concat([gdf_streams, manual_gdf], ignore_index=True)
merged_gdf = gpd.GeoDataFrame(merged_gdf, crs=gdf_streams.crs)

# Save
merged_gdf.to_file(out_fp)
print(f"✅ Merged shapefile saved: {out_fp}")

✅ Merged shapefile saved: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5.shp


/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'arcid' to 'arcid_1'
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field created_da create as date field, though DateTime requested.
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field last_edite create as date field, though DateTime requested.
  ogr_write(


In [2]:
import tools  # your custom GIS functions
import os

dem_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif'

streams_fp = [
    r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5.shp',
    r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams.shp'
]

burn_fields = ['VALUE', 'GRID_CODE']

for stream_file, burn_field in zip(streams_fp, burn_fields):
    base_name = os.path.splitext(os.path.basename(stream_file))[0]
    out_fp = os.path.join(os.path.dirname(stream_file), f'{base_name}_0.5m.tif')
    
    tools.rasterize_shapefile(
        shapefile=stream_file,
        burn_field=burn_field,  # use the corresponding attribute
        subset=None,
        out_fp=out_fp,
        ref_raster=dem_path,
        plot=False,
        save_in='tif'
    )
    
    print(f"✅ Rasterized {stream_file} using {burn_field} -> {out_fp}")

✅ Rasterized /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5.shp using VALUE -> /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5_0.5m.tif
✅ Rasterized /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams.shp using GRID_CODE -> /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams_0.5m.tif


In [3]:
import tools  # your custom GIS functions
import os
import rasterio
import numpy as np

dem_path = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif'

streams_fp = [
    r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5.shp',
    r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams.shp'
]

burn_fields = ['VALUE', 'GRID_CODE']

for stream_file, burn_field in zip(streams_fp, burn_fields):
    base_name = os.path.splitext(os.path.basename(stream_file))[0]
    mask_fp = os.path.join(os.path.dirname(stream_file), f'{base_name}_0.5m.tif')
    elev_fp = os.path.join(os.path.dirname(stream_file), f'{base_name}_0.5m_elev.tif')

    # Step 1: Rasterize the shapefile
    tools.rasterize_shapefile(
        shapefile=stream_file,
        burn_field=burn_field,  # use the corresponding attribute
        subset=None,
        out_fp=mask_fp,
        ref_raster=dem_path,
        plot=False,
        save_in='tif'
    )
    print(f"✅ Rasterized {stream_file} using {burn_field} -> {mask_fp}")

    # Step 2: Read DEM and rasterized mask
    with rasterio.open(dem_path) as dem_src, rasterio.open(mask_fp) as mask_src:
        dem = dem_src.read(1)
        mask = mask_src.read(1)
        meta = mask_src.meta

        nodata_val = -9999  # define nodata for elevation raster
        #print(nodata_val)

        # Step 3: Assign DEM elevations only where mask is 1
        elev_raster = np.full_like(dem, nodata_val, dtype=np.float32)
        elev_raster[mask == 1] = dem[mask == 1]

        # Step 4: Prepare metadata for elevation raster
        elev_meta = mask_src.meta.copy()
        elev_meta.update(dtype=rasterio.float32, nodata=nodata_val)

    # Step 5: Save elevation raster
    with rasterio.open(elev_fp, 'w', **elev_meta) as dst:
        dst.write(elev_raster, 1)

    print(f"✅ Saved DEM elevations at stream locations -> {elev_fp}")

✅ Rasterized /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5.shp using VALUE -> /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5_0.5m.tif
✅ Saved DEM elevations at stream locations -> /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5_0.5m_elev.tif
✅ Rasterized /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams.shp using GRID_CODE -> /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams_0.5m.tif
✅ Saved DEM elevations at stream locations -> /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/